# 1.1 Serving a Private Model (GPT-2)
**protecting model owner's intellectual property (IP)**

## Dependencies

Make you sure you have [pytorch-transformers](https://github.com/huggingface/pytorch-transformers) installed. PyTorch-Transformers can be installed by pip as follows:

In [1]:
# !pip install pytorch-transformers

In [2]:
import torch as th
import grid as gr
import syft as sy
import gpt2_utils

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# GPT-2 model
from pytorch_transformers import GPT2LMHeadModel

In [3]:
hook = sy.TorchHook(th)

## Connect to a GridWorker

In [4]:
ai_inc = gr.WebsocketGridClient(hook, "http://localhost:3000/", id="ai_inc")
ai_inc.connect()

## Model and Data Setup

In [5]:
# Load pre-trained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2',torchscript=True)

In [6]:
# Encode a text inputs
text = "Who was Jim Henson ? Jim Henson was a"
tokens_tensor = gpt2_utils.encode_text(text)

IMPORTANT: if you trace the model with a tensor of size `N` the trace model only expects inputs of size `N`.

In [7]:
traced_model = th.jit.trace(model, (tokens_tensor,))

## Hosting a Private Model

The important part is that we set `allow_remote_inference=True` so a user can run inference in the model hosted on this worker.

In [8]:
ai_inc.serve_model(traced_model, model_id="GPT-2-private", allow_remote_inference=True, allow_download=False)

True